In [2]:
import numpy as np
import pandas as pd
import sys
from numpy.linalg import inv, pinv
from numpy import dot
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

In [17]:
csv = pd.read_csv('X_train.csv', header=None, names=['X','Y'])
t = pd.read_csv('T_train.csv', header=None, names=['H'])

In [18]:
csv['target'] = t['H']

In [19]:
csv = csv.sample(frac=1)

In [20]:
validation, train = np.split(csv.sample(frac=1), [len(csv)//5])

In [21]:
def n_fold_train_val_split():
    train_val = dict()
    for i in range(5):
        try:
            val, tr = (csv.iloc[i*int(0.2*len(csv)): (i+1)*int(0.2*len(csv))], 
                          csv.drop(range(i*int(0.2*len(csv)), (i+1)*int(0.2*len(csv)))))
            train_val[i] = (tr, val)
        except IndexError:
            this_fold = (csv.iloc[i*int(0.2*len(csv)): len(csv)], 
                         csv.drop(range(i*int(0.2*len(csv)), len(csv))))
            train_val[i] = (tr, val)

In [22]:
validation = validation[:200]
train = train[:1000]

In [23]:
class Gaussian2D():
    def __init__(self, mean_x, mean_y, variance_x=1, variance_y=1):
        self.mean_x = mean_x
        self.mean_y = mean_y
        self.variance_x = variance_x
        self.variance_y = variance_y
        self.prob = None
        
    def probability(self, x, y):
        self.prob = (np.exp(-((x-self.mean_x)**2)/(2*self.variance_x**2) - 
                             ((y-self.mean_y)**2)/(2*self.variance_y**2)))
        return self.prob
    
    def __str__(self):
        return 'Gaussian2D with mean {}, {}; variance {}, {}'.format(self.mean_x, self.mean_y, self.variance_x, self.variance_y)

In [24]:
# multiple regression
# 座標點值1~1081, 切grid
# 假設都由高斯分布組成
# mean為坐標點, var隨意設
feature_basis = [1] # 1: bias
for i in range(200, 1082, 1):
    for j in range(1, 1082, 1):
        feature_basis.append(Gaussian2D(i, j, 10, 10))
for i in range(1, 200, 50):
    for j in range(1, 1082, 50):
        feature_basis.append(Gaussian2D(i, j, 10, 10))

In [25]:
feature_basis = [1]
for index, row in train.iterrows():
    feature_basis.append(Gaussian2D(row['X'], row['Y'], 0.1, 0.1))

In [26]:
def create_design_matrix(data, feature_basis):
    design_matrix = np.zeros((len(data), len(feature_basis)))
    length = design_matrix.shape[1]
    for i in range(1, length): # by column, 0th is basis, skip
        design_matrix[:,i] = data.apply(lambda row: feature_basis[i].probability(row['X'], row['Y']), axis=1)
        done = 80 * i // length
        sys.stdout.write("\r[%s%s]" % ('=' * done, ' ' * (80-done)))
    return design_matrix

In [27]:
design_matrix = create_design_matrix(train, feature_basis)
design_matrix

[=============================================================================== ]

array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [29]:
W_ML = dot(pinv(design_matrix), train['target'])
W_ML

array([ -3.42496567e-14,   0.00000000e+00,   0.00000000e+00, ...,
         2.86000000e+02,   1.60000000e+01,   0.00000000e+00])

In [30]:
def evaluate(W_ML, feature_basis, test_data, design_matrix = None):
    if design_matrix is None:
        design_matrix = create_design_matrix(test_data, feature_basis)
    for i in range(len(W_ML)):
        design_matrix[:,i] *= W_ML[i]
    err = 0
    for i in range(len(design_matrix)):
        err += (np.sum(design_matrix[i:]) - test_data['target'].iloc[i])**2
    err /= len(design_matrix)
    return err

In [31]:
evaluate(W_ML, feature_basis, train, design_matrix)

2434257143.348

In [37]:
# 先假設為multivariate gaussian
mu = 1/len(train)*(train[['X', 'Y']].sum())

In [71]:
cov = np.zeros((2,2))
for row in np.array(train[['X', 'Y']]-mu):
    cov += np.dot(row.reshape(2,1), row.reshape(2,1).T)
cov /= len(train)